In [1]:
import os
import csv
import datetime, pytz
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps
from edc_base.utils import get_utcnow
from edc_constants.constants import OTHER

report_dt = datetime.datetime(2023, 2, 16, 8, 10)
tz = pytz.timezone('Africa/Gaborone')
report_dt = tz.localize(report_dt)

from flourish_caregiver.helper_classes import CaregiverBiologicalSwitch

subject_identifier='C142-040990903-2'
switch_cls = CaregiverBiologicalSwitch(caregiver_sid=subject_identifier)

In [2]:
"""
    Take caregiver offstudy
"""
switch_cls.take_caregiver_offstudy(
    report_dt=report_dt,
    offstudy_dt=report_dt.date(),
    reason='caregiver_death', )

True

In [2]:
"""
    Create a locator instance for the caregiver taking over, with new screening identifier
    and update maternal dataset object with identifier.   
"""
switch_cls.create_bio_mother_locator(
    report_dt=report_dt, signed_dt=report_dt.date(), first_name='NEW', last_name='CAREGIVER')

In [3]:
switch_cls.screening_identifier

'S99E7MC2'

In [4]:
"""
    Create an instance of the BHP prior screening for the mother
"""
from dateutil.relativedelta import relativedelta
from edc_constants.constants import NOT_APPLICABLE, YES, NO, OTHER
screening_options = {
    'child_alive': YES,
    'mother_alive': NO,
    'flourish_participation': 'another_caregiver_interested'}
switch_cls.create_bio_screening(report_dt=report_dt, **screening_options)

<ScreeningPriorBhpParticipants: S99E7MC2, 056-4995336-3>

In [5]:
"""
    Consent the mother to the study
"""
consent_options = {'language': 'en',
                    'recruit_source': 'BHP recruiter/clinician',
                    'recruitment_clinic': 'Prior',
                    'is_literate': YES,
                    'dob': get_utcnow() - relativedelta(years=26),
                    'is_dob_estimated': NO,
                    'citizen': YES,
                    'identity': '224422211',
                    'identity_type': 'country_id',
                    'confirm_identity': '224422211',
                    'remain_in_study': YES,
                    'hiv_testing': NOT_APPLICABLE,
                    'breastfeed_intent': NOT_APPLICABLE,
                    'child_consent': YES,
                    'future_contact': YES,
                    'consent_reviewed': YES,
                    'study_questions': YES,
                    'assessment_score': YES,
                    'consent_signature': YES,
                    'consent_copy': YES,
                    'consent_datetime': report_dt}
switch_cls.create_bio_consent(sid_swap=('C', 'CR'), **consent_options)

In [6]:
"""
    Add the child consent to the biological mother, removing it from the caregiver
"""
switch_cls.add_child_consent_to_mother()

In [7]:
"""
    Update biological mother's ID to the child's registered subject object
"""
switch_cls.update_child_registered_subject()

In [8]:
"""
    Create instance of the biological mother's previous enrollment information
"""
prev_enrol_defaults = {'maternal_prev_enroll': NO, 'relation_to_child': 'aunt'}
switch_cls.create_bio_previous_enrol_info(report_dt=report_dt, **prev_enrol_defaults)

<CaregiverPreviouslyEnrolled: CR142-040990903-2>

In [9]:
"""
    Put mother on the enrolment schedule
"""
import datetime, pytz
onschedule_dt = datetime.datetime(2023, 2, 16, 10, 0)
tz = pytz.timezone('Africa/Gaborone')
onschedule_dt = tz.localize(onschedule_dt)
switch_cls.put_on_enrol_schedule(onschedule_dt=onschedule_dt)

In [10]:
"""
    Put mother on the quarterly schedule, and align window periods to child appointments
"""
switch_cls.put_on_quart_schedule(onschedule_dt=onschedule_dt)

In [11]:
switch_cls.align_with_child_appts()